<a href="https://colab.research.google.com/github/xqwu1108-create/MSE1003H_XueqiuWu_1011808741/blob/main/A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Introduction**

# **Analysis**

# **1. Dataset Research and Loading**

In [15]:
import numpy as np
import pandas as pd
import os
from mp_api.client import MPRester
from pymatgen.core import Composition

In [14]:
import os
print(os.getenv("MP_API_KEY"))



My_API_Key
